In [1]:
#Import a few libraries
import glob
from pathlib import Path
import os
import sys
import numpy as np
from scipy.io import loadmat

In [2]:
#Path setup
#Note that appending path p, which is where MAPIT lies,
#is required if MAPIT is not installed as a python package
#from pip/conda

#Also adjust the path for analysis data, F
#if needbe. This assumes csv data which
#is derived from the included dataset
x = Path(os.getcwd()).resolve().parents[0]
p = Path(os.getcwd()).resolve().parents[1]
sys.path.append(str(p))
F = os.path.join(x, 'debug_data','csv')

In [3]:
from MAPIT.core import Preprocessing, StatsTests, AuxFunctions

---

<div align="center">

#  Data import 
### The following blocks of code will change based on individual use cases

</div>



---

In [4]:
#This sets a path object for the 
#inputs, inventories, and outputs
indir = os.path.join(F,'input')
invdir = os.path.join(F,'inventory')
outdir = os.path.join(F,'output')


In [5]:
#Use glob to get list of files
#For each measurement type
infiles = glob.glob(indir+'/*.csv')
invfiles = glob.glob(invdir+'/*.csv')
outfiles = glob.glob(outdir+'/*.csv')

In [6]:
#This just makes sure that the matrix shape
#of the csv data is 
# samples, features (time, values for MAPIT)
def checkImportAxes(d):
  if d.shape[0] < d.shape[1]:
    np.swapaxes(d,0,1)
  
  return d

In [7]:
#Read in the csv data
#And store it in a list (required)
indat = []
invdat = []
outdat = []

inTdat = []
invTdat = []
outTdat = []

for i in range(len(infiles)):
    z = checkImportAxes(np.loadtxt(infiles[i],delimiter=','))
    inTdat.append(z[:,0])
    indat.append(z[:,1])

for i in range(len(invfiles)):
    z = checkImportAxes(np.loadtxt(invfiles[i],delimiter=','))
    invTdat.append(z[:,0])
    invdat.append(z[:,1])

for i in range(len(outfiles)):
    z = checkImportAxes(np.loadtxt(outfiles[i],delimiter=','))
    outTdat.append(z[:,0])
    outdat.append(z[:,1])

In [8]:
#alternative loading showing how you might load .mat data
#largely based on SSPM-based measurement blocks
#note the squeeze_me
#TODO: upload data from desktop
# F = os.path.join(x, 'debug_data','csv')
# x1 = loadmat(F,squeeze_me=True)

# invdat = x1['invn']['data']
# inpdatt = x1['in']['data']
# outdat = x1['outn']['data']

# invTdat = x1['invn']['time']
# inTdat = x1['in']['time']
# outTdat = x1['outn']['time']

#### Note that all of the data and time variables are lists that contain arrays. This is because MAPIT supports variable sampling so there might be different numbers of samples for different locations. However, the data, time array pairs should be the same length at each location. For example ``indat[0].shape = inTdat[0].shape``

In [9]:
#Note here that all the data and time
#variables are lists that contain arrays,
#because MAPIT supports variable sampling,  
print(type(indat))
print(type(invdat))
print(type(outdat))

print(type(inTdat))
print(type(invTdat))
print(type(outTdat))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


The next block specifies relative standard deviation which is stored in `ErrorMatrix` with `(nlocs,2)`. `nlocs` reflects the total number of locations at each location. The first value in the second dimension (e.g., `ErrorMatrix[:,0]`) refers to the random error whereas the second entry (e.g., `ErrorMatrix[:,1]`) refers to the systematic error.

Important: ErrorMatrix expects relative standard deviation (RSD) in decimal form, *not* percentage. For example, a 10% relative standard deviation should be specified as 0.10 

Note that for this example, a uniform RSD is specified at 0.5% for all locations

A `TotalErrorMatrix` is also specified, which is required for certain statistical operations. It is expected that the locations will be stacked as (inputs, inventories, outputs)


In [10]:
inputErrorMatrix = np.ones((len(indat),2)) * 0.005 
inventoryErrorMatrix = np.ones((len(invdat),2)) * 0.005 
outputErrorMatrix = np.ones((len(outdat),2)) * 0.005 
TotalErrorMatrix = np.vstack((inputErrorMatrix,inventoryErrorMatrix,outputErrorMatrix))
IT = 25
mbaTime = 416

In [11]:
processedInput, processedInputTimes, \
processedInventory, processedInventoryTimes, \
processedOutput, processedOutputTimes = Preprocessing.FormatInput(
                                            rawInput = indat,
                                            rawInventory = invdat,
                                            rawOutput = outdat,
                                            rawInputTimes = inTdat,
                                            rawInventoryTimes = invTdat,
                                            rawOutputTimes = outTdat)

In [12]:
MBPs = AuxFunctions.getMBPs(processedInputTimes,processedInventoryTimes,processedOutputTimes,mbaTime)

In [13]:
inputAppliedError = Preprocessing.SimErrors(
                            rawData = processedInput, 
                            ErrorMatrix =  inputErrorMatrix, 
                            iterations = IT)

inventoryAppliedError = Preprocessing.SimErrors(
                            rawData = processedInventory,
                            ErrorMatrix =  inventoryErrorMatrix, 
                            iterations = IT)


outputAppliedError = Preprocessing.SimErrors(
                            rawData = processedOutput, 
                            ErrorMatrix =  outputErrorMatrix, 
                            iterations = IT)

100.00% |██████████|  [Elapsed: 00:00 || Remaining: 00:00]
100.00% |██████████|  [Elapsed: 00:00 || Remaining: 00:00]
100.00% |██████████|  [Elapsed: 00:01 || Remaining: 00:00]


In [14]:
print(type(indat))
print(type(invdat))
print(type(outdat))

print(processedInput[0].shape == inputAppliedError[0].shape[1:])

<class 'list'>
<class 'list'>
<class 'list'>
True


In [15]:
inputAppliedError[0].shape

(25, 130049, 1)

In [16]:
# test case w/no applied error
# inputAppliedError = processedInput
# inventoryAppliedError = processedInventory
# outputAppliedError = processedOutput

In [17]:
MUF = StatsTests.MUF(inputAppliedError = inputAppliedError,
                            inventoryAppliedError= inventoryAppliedError,
                            outputAppliedError = outputAppliedError,
                            processedInputTimes = processedInputTimes,
                            processedInventoryTimes = processedInventoryTimes,
                            processedOutputTimes = processedOutputTimes,
                            MBP = mbaTime)

100.00% |██████████|  [Elapsed: 00:01 || Remaining: 00:00]


In [18]:
CUMUF = StatsTests.CUMUF(MUF=MUF)

100.00% |██████████|  [Elapsed: 00:00 || Remaining: 00:00]


In [19]:
SEMUF, _, _, _ = StatsTests.SEMUF(inputAppliedError = inputAppliedError,
                                    inventoryAppliedError= inventoryAppliedError,
                                    outputAppliedError = outputAppliedError,
                                    processedInputTimes = processedInputTimes,
                                    processedInventoryTimes = processedInventoryTimes,
                                    processedOutputTimes = processedOutputTimes,
                                    MBP = mbaTime,
                                    ErrorMatrix = TotalErrorMatrix)

100.00% |██████████|  [Elapsed: 00:01 || Remaining: 00:00]


In [20]:
SITMUF = StatsTests.SITMUF(inputAppliedError = inputAppliedError,
                                                    inventoryAppliedError= inventoryAppliedError,
                                                    outputAppliedError = outputAppliedError,
                                                    processedInputTimes = processedInputTimes,
                                                    processedInventoryTimes = processedInventoryTimes,
                                                    processedOutputTimes = processedOutputTimes,
                                                    ErrorMatrix = TotalErrorMatrix,
                                                    MBP = mbaTime,
                                                    MUF = MUF) 

1.67% |▏         |  [Elapsed: 00:17 || Remaining: 00:06]

In [21]:

Page = StatsTests.PageTrendTest(SITMUF,mbaTime,MBPs)

100%|██████████| 375/375 [00:00<00:00, 144180.40it/s]
